In [30]:
import joblib

# Load the model from the file
model = joblib.load('model.joblib')


In [34]:
import pickle

# Load the pickle file
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Verify the loaded model
print(loaded_model)

EOFError: Ran out of input

In [32]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Load the data from synthetic_data.csv
data = pd.read_csv('sampled_data.csv')

# Assuming the last column is the target variable
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Predict using the loaded model
y_pred = model.predict(X)

# Evaluate the model
accuracy = accuracy_score(y, y_pred)
print(f'Accuracy: {accuracy}')

/Users/sanjanathyady/Desktop/project-2/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: X has 9 features, but RandomForestClassifier is expecting 6 features as input.

In [6]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

import matplotlib.pyplot as plt

# Generate the confusion matrix
cm = confusion_matrix(y, y_pred)

cm

array([[    0,  1417],
       [    0, 58583]])

In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Step 1: Load Dataset
csv_path = "dataset.csv"  # Replace with your dataset file
data = pd.read_csv(csv_path)

# Step 2: Feature Engineering
def hash_ip(ip):
    # Simple hash function for IP addresses
    return hash(ip) % (10**9)

def hash_protocol(protocol):
    # Convert protocol to hashable integer
    return hash(protocol)

# Apply transformations to relevant columns
data['src_ip_hash'] = data['IPV4_SRC_ADDR'].apply(hash_ip)
data['dst_ip_hash'] = data['IPV4_DST_ADDR'].apply(hash_ip)
data['protocol_hash'] = data['PROTOCOL'].apply(hash_protocol)

# Step 3: Select Relevant Features
selected_features = [
    "src_ip_hash", "dst_ip_hash", "protocol_hash",
    "IN_BYTES", "OUT_BYTES", "IN_PKTS", "OUT_PKTS",
    "TCP_FLAGS", "FLOW_DURATION_MILLISECONDS", "Label"
]
data = data[selected_features]

# Step 4: Handle Missing Values
data = data.dropna()  # Drop rows with missing values

# Step 5: Normalize Numerical Features to [0, 1]
numerical_features = [
    "src_ip_hash", "dst_ip_hash", "protocol_hash",
    "IN_BYTES", "OUT_BYTES", "IN_PKTS", "OUT_PKTS",
    "TCP_FLAGS", "FLOW_DURATION_MILLISECONDS"
]
scaler = MinMaxScaler(feature_range=(0, 1))
data[numerical_features] = scaler.fit_transform(data[numerical_features])

# Step 6: Encode Labels
label_encoder = LabelEncoder()
data["Label"] = label_encoder.fit_transform(data["Label"])  # Convert labels to integers



In [8]:
data.head()


,src_ip_hash,dst_ip_hash,protocol_hash,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label
0,0.594721,0.935978,0.087719,5.146556e-04,0.000210,0.000428,0.000363,0.107623,0.999997,1
1,0.594721,0.028459,0.087719,5.286060e-06,0.000217,0.000083,0.000158,0.107623,0.999997,0
2,0.028539,0.594917,0.000000,6.889101e-07,0.000000,0.000002,0.000000,0.000000,0.000000,0
3,0.314920,0.670330,0.280702,5.332429e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,0.682512,0.211134,0.280702,1.954120e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [10]:
df_label_0 = data[data['Label'] == 0]
df_label_0

,src_ip_hash,dst_ip_hash,protocol_hash,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label
1,0.594721,0.028459,0.087719,5.286060e-06,2.173117e-04,0.000083,0.000158,0.107623,0.999997,0
2,0.028539,0.594917,0.000000,6.889101e-07,0.000000e+00,0.000002,0.000000,0.000000,0.000000,0
3,0.314920,0.670330,0.280702,5.332429e-07,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0
4,0.682512,0.211134,0.280702,1.954120e-07,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0
6,0.935617,0.594917,0.000000,1.039989e-06,0.000000e+00,0.000004,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...
16940491,0.028539,0.594917,0.087719,1.028828e-04,7.053010e-06,0.000102,0.000071,0.107623,0.000000,0
16940492,0.339495,0.840759,0.087719,1.414584e-05,1.799733e-06,0.000009,0.000012,0.112108,0.000000,0
16940493,0.172842,0.594917,0.087719,6.193567e-07,9.181073e-07,0.000006,0.000007,0.112108,0.000000,0
16940494,0.078284,0.594917,0.280702,2.252206e-07,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0


In [12]:
# Divide df_label_0 into chunks of 10,000 rows each
chunks = [df_label_0.iloc[i:i + 10000] for i in range(0, len(df_label_0), 10000)]

# Verify the number of chunks
len(chunks)

610

In [13]:
chunks[0]

,src_ip_hash,dst_ip_hash,protocol_hash,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label
1,0.594721,0.028459,0.087719,5.286060e-06,0.000217,0.000083,0.000158,0.107623,0.999997,0
2,0.028539,0.594917,0.000000,6.889101e-07,0.000000,0.000002,0.000000,0.000000,0.000000,0
3,0.314920,0.670330,0.280702,5.332429e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,0.682512,0.211134,0.280702,1.954120e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0
6,0.935617,0.594917,0.000000,1.039989e-06,0.000000,0.000004,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...
10155,0.102150,0.287200,0.087719,1.324827e-07,0.000000,0.000000,0.000000,0.008969,0.000000,0
10156,0.102150,0.287200,0.087719,1.324827e-07,0.000000,0.000000,0.000000,0.008969,0.000000,0
10157,0.102150,0.840759,0.087719,1.324827e-07,0.000000,0.000000,0.000000,0.008969,0.000000,0
10158,0.102150,0.287200,0.087719,1.324827e-07,0.000000,0.000000,0.000000,0.008969,0.000000,0


In [15]:
# Concatenate all chunks into a single DataFrame
all_chunks = pd.concat(chunks)

# Save the concatenated DataFrame to a single CSV file
all_chunks.to_csv('all_chunks.csv', index=False)

In [16]:
# Sample 10% of the data
sampled_data = data.sample(frac=0.1, random_state=42)

# Verify the size of the sampled data
sampled_data.shape
# Save the sampled data to a new CSV file
sampled_data.to_csv('sampled_data.csv', index=False)

In [18]:
data = pd.read_csv('dataset.csv')
